# Kinesis

Create Kinesis Stream and query information

In [ ]:
%%time
import os, boto3, uuid, time, json
import inspect
import zipfile, os, stat

boto3.setup_default_session(profile_name='dev')

# Configuration
iam_role_name='lambda-streaming-'
unique_id = str(uuid.uuid1())[:5]
stream_name = "stream-"+ unique_id
function_name = "stream-processing-"+ unique_id

In [ ]:
%%time
kinesis_client = boto3.client('kinesis', region_name='us-east-1')
response = kinesis_client.create_stream(
    StreamName=stream_name,
    ShardCount=1
)
print("Created stream: %s"%stream_name)
stream_arn=kinesis_client.describe_stream(StreamName=stream_name)['StreamDescription']['StreamARN']
while kinesis_client.describe_stream(StreamName=stream_name)['StreamDescription']['StreamStatus']!='ACTIVE':
    time.sleep(1)
    
stream_arn

In [ ]:
kinesis_client.describe_stream(StreamName=stream_name)

In [ ]:
put_response = kinesis_client.put_record(
                        StreamName=stream_name,
                        Data="Hello World",
                        PartitionKey="A")

In [ ]:
response = kinesis_client.delete_stream(StreamName=stream_name)

# Lambda



## Prepare Source Code for Lambda with Inspect

In [ ]:
import stat
def prepare_file(function_ref, file_basename):
    print("Create function: %s"%file_basename)
    lines = inspect.getsource(function_ref)
    with open(file_basename + ".py", "w") as f:
        f.write(lines)
    os.chmod(file_basename + ".py", stat.S_IRUSR | stat.S_IWUSR | stat.S_IRGRP | stat.S_IROTH)
    with zipfile.ZipFile(file_basename + '.zip', 'w') as myzip:
        myzip.write(file_basename + ".py")
    env_variables = dict() # Environment Variables
    with open(file_basename + '.zip', 'rb') as f:
        zipped_code = f.read()
    
    os.remove(file_basename + '.zip')
    os.remove(file_basename + '.py')
    return zipped_code

In [ ]:
def lambda_handler(event, context):
    import base64
    print("Lambda Function called")
    for record in event['Records']:
        #Kinesis data is base64 encoded so decode here
        payload=base64.b64decode(record["kinesis"]["data"])
        print("Decoded payload: " + str(payload))

In [ ]:
zipped_code=prepare_file(lambda_handler, function_name)

## Setup Lambda

Create Lambda Function

In [ ]:
def create_lambda_iam_role(rolename):
    iam_client = boto3.client('iam')
    role_policy_document = {
      "Version": "2012-10-17",
      "Statement": [
        {
          "Sid": "",
          "Effect": "Allow",
          "Principal": {
            "Service": "lambda.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
      ]
    }
    iam_client.create_role(
      RoleName=rolename,
      AssumeRolePolicyDocument=json.dumps(role_policy_document),
    )
    role = iam_client.get_role(RoleName=rolename)
    response = iam_client.attach_role_policy(
        RoleName=rolename,
        PolicyArn='arn:aws:iam::aws:policy/AmazonKinesisFullAccess'
    )
    response = iam_client.attach_role_policy(
        RoleName=rolename,
        PolicyArn='arn:aws:iam::aws:policy/CloudWatchLogsFullAccess'
    )
    
    
    return role['Role']['Arn']
role_arn = create_lambda_iam_role(iam_role_name)

In [ ]:
lambda_client = boto3.client('lambda', region_name='us-east-1')
response = lambda_client.create_function(
    FunctionName=function_name,
    Runtime='python3.7',
    Role=role_arn,
    Handler=function_name+'.lambda_handler',
    Code={
        'ZipFile': zipped_code
    },
    Description='string'
    
)
lambda_client.get_function(
    FunctionName=function_name
)

Create Event-Source Mapping to Kinesis Stream

In [ ]:
print("Create mapping from %s to %s"%(stream_arn, function_name))
response = lambda_client.create_event_source_mapping(
    EventSourceArn=stream_arn,
    FunctionName=function_name,
    Enabled=True,
    BatchSize=1,
    StartingPosition='LATEST'
)

In [ ]:
print("Put Message to Stream: " + stream_name)
put_response = kinesis_client.put_record(
                        StreamName=stream_name,
                        Data="Hello World 4",
                        PartitionKey="A")
put_response

In [ ]:
response = lambda_client.delete_function(
    FunctionName=function_name
)

## Other Lambda Management Code

In [ ]:
lambda_client.invoke(
  FunctionName=function_name,
  InvocationType='Event',
  Payload="{}",
)

In [ ]:
response = lambda_client.update_function_code(
    FunctionName=function_name,
    ZipFile=prepare_file(lambda_handler, function_name),
    RevisionId='883c29c6-2e06-474f-b89c-5d0adf425f28'
)

# Cleanup

In [ ]:
stream_names=kinesis_client.list_streams()['StreamNames']
for i in stream_names:
    kinesis_client.delete_stream(StreamName=i)

function_names=lambda_client.list_functions()['Functions']
for i in function_names:
    lambda_client.delete_function(FunctionName=i['FunctionName'])

only lambda

In [ ]:
function_names=lambda_client.list_functions()['Functions']
for i in function_names:
    lambda_client.delete_function(FunctionName=i['FunctionName'])

only iam rol

In [ ]:
iam_client.detach_role_policy(RoleName=iam_role_name, PolicyArn='arn:aws:iam::aws:policy/AmazonKinesisFullAccess')
iam_client.detach_role_policy(RoleName=iam_role_name, PolicyArn='arn:aws:iam::aws:policy/CloudWatchLogsFullAccess')
iam_client.delete_role(RoleName=iam_role_name)

# Scratch

## Pickle Serialization

In [ ]:
import pickle
serialized_function=pickle.dumps(hello_world)

In [ ]:
f=pickle.loads(serialized_function)
f()

In [ ]:
with open("lambda_test.py", "w") as f:
    f.write("""
import pickle

f=pickle.loads(%s)
f()
"""%serialized_function)
    

## Joblib

In [ ]:
import joblib, io
with open("hello.jl", 'wb') as f:
    joblib.dump(hello_world, f)

In [ ]:
with open("hello.jl", 'rb') as f:
    f=joblib.load(f)

In [ ]:
f()

In [ ]:
output = io.BytesIO()
hello_world.__module__=="__main__"
joblib.dump(hello_world, output)

In [ ]:
output.getvalue()

In [ ]:
deser_output = io.BytesIO()
deser_output.write(output.getvalue())
f=joblib.load(deser_output)

In [ ]:
f()

In [ ]:
with open("tmp_lambda_test_job.py", "w") as f:
    f.write("""
import joblib
deser_output = io.BytesIO()
deser_output.write(%s)
f=joblib.load(deser_output)
f()
"""%output.getvalue())

## Dill

In [ ]:
import dill
o=dill.dumps(hello_world)

In [ ]:
dill.loads(o)()

In [ ]:
with open("tmp_lambda_test_job.py", "w") as f:
    f.write("""import dill
f=dill.loads(%s)
f()
"""%o)